In [1]:
using Expectations, ForwardDiff, QuadGK, LinearAlgebra, FastGaussQuadrature

## Adaptive Quadrature
Gauss-Kronod is very precise, changes step sizes

In [2]:
f(x) = x^2
x_min = 0.0
x_max = 1.0
x = x_min:0.01:x_max
x_array = Array(x)  # turns into an array.  Doesn't need to be uniform anymore
@show quadgk(f, x_min, x_max)[1]

(quadgk(f, x_min, x_max))[1] = 0.3333333333333333


0.3333333333333333

## Trapezoidal Rule (uniform and non-uniform)
$$ \int_{x_{\min}}^{x_{\max}} f(x) dx \approx \sum_{i=1}^N f(x_i) w_i = f(\vec{x}) \cdot w $$

In [3]:
function trapz_integral(f, nodes::AbstractRange)
    M = length(nodes)
    Δ = step(nodes)
    total = zero(eltype(nodes))
    for (i, x) in enumerate(nodes)
        weight = ((i == 1) || (i == M)) ?  Δ/2 : Δ   # ternary operation, Condition ? if true : if false
        total += weight * f(x)
    end
    return total
end

trapz_integral (generic function with 1 method)

In [5]:
trapz_integral(f, x)

0.33335000000000015

In [6]:
function trapz_weights(nodes::AbstractArray)  # Note multiple dispatch
    println("trapz for AbstractArray:")
    M = length(nodes)
    Δ = diff(nodes)
    prepend!(Δ, NaN) # To keep the indexing straight. Now, Δ[2] = Δ_2 = z_2 - z_1. And NaN will throw an error if we try to use it.
    interiorWeights = [(Δ[i] + Δ[i+1])/2 for i = 2:M-1]
    return [Δ[2]/2; interiorWeights; Δ[M]/2]
end

trapz_weights (generic function with 1 method)

In [7]:
w = trapz_weights(x_array)  # call with array
@show w ⋅ f.(x_array);

trapz for AbstractArray:
w ⋅ f.(x_array) = 0.33335


In [9]:
w = trapz_weights(x) # call with range
@show w ⋅ f.(x);

trapz for AbstractArray:
w ⋅ f.(x) = 0.33335


In [10]:
# Specialization.  New method added to `trapz_weights`
function trapz_weights(nodes::AbstractRange)
    println("trapz for AbstractRange:")
    M = length(nodes)
    Δ = step(nodes)
    return [Δ/2; Δ*ones(M-2); Δ/2]
end

trapz_weights (generic function with 2 methods)

In [11]:
w = trapz_weights(x)
@show w ⋅ f.(x);

trapz for AbstractRange:
w ⋅ f.(x) = 0.33335000000000004


## Example: Legendre Quadrature
See https://en.wikipedia.org/wiki/Gaussian_quadrature#Gauss%E2%80%93Legendre_quadrature
$$ \int_{-1}^1 f(x) dx \approx \sum_{i=1}^N f(x_i) w_i = f(\vec{x}) \cdot w $$


In [14]:
N = 10
x, w = gausslegendre(N)  # can adjust to change range
@show quadgk(f, -1.0, 1.0)[1]
dot(w, f.(x)) # note few evaluations

(quadgk(f, -1.0, 1.0))[1] = 0.6666666666666667


0.666666666666667